In [1]:
user = "admin"
passw = "AsH3saUzNX"
uri = "wheeliegood.cpd1upycyviv.us-east-1.rds.amazonaws.com"
# APIKEY = "faa0b3cb4ae90aba340567484b057e59c72b1a37"
APIKEY = "937848f62521aa822495dbe1716d5ca9ecf84fba"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"
WEATHER_INFO = "https://api.openweathermap.org/data/2.5/weather"
FORECAST_INFO = "https://api.openweathermap.org/data/2.5/onecall"
FORECAST_API = "5cf5e19adac515b3f8510d2692fddc43"
WAPIKEY = "4f52f429445505a8554ffaf9b514990d"
WNAME = "2964574"
#https://api.openweathermap.org/data/2.5/onecall?lat=53.33306&lon=-6.24889&appid=5cf5e19adac515b3f8510d2692fddc43

In [2]:
from sqlalchemy import create_engine
from pandas import pandas as pd
from functools import lru_cache
from sklearn.tree import DecisionTreeClassifier
import requests
import json
import datetime

In [3]:

# def current_bikes():
#     # Request Data from API
#     engine = create_engine(f"mysql+mysqlconnector://{user}:{passw}@{uri}:3306/wheelieGood",
#                            echo=True)
#     # Using static bike table
#     df = pd.read_sql('SELECT number, available_bike_stands, available_bikes, '
#                      'MAX(last_update) FROM wheelieGood.dynamic_bikes dynB GROUP BY number;', engine)
#     dynamic_bike_data = df.to_json(orient="records")
#     return dynamic_bike_data

def dynamic_bikes():
    # Request Data from API
    engine = create_engine(f"mysql+mysqlconnector://{user}:{passw}@{uri}:3306/wheelieGood",
                           echo=True)
    # Using static bike table
    bike_data = pd.read_sql("SELECT * FROM wheelieGood.dynamic_bikes;", engine)
    return bike_data

def dynamic_weather():
    # Request Data from API
    engine = create_engine(f"mysql+mysqlconnector://{user}:{passw}@{uri}:3306/wheelieGood",
                           echo=True)
    # Using static bike table
    weather_data = pd.read_sql("SELECT * from weather", engine)
    return weather_data



In [4]:
weather = dynamic_weather()
bikes = dynamic_bikes()

bikes.head(10)

2021-04-01 00:06:32,444 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-04-01 00:06:32,447 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:32,808 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-04-01 00:06:32,809 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:33,545 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-04-01 00:06:33,545 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:34,252 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-04-01 00:06:34,253 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:34,622 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-04-01 00:06:34,623 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:35,108 INFO sqlalchemy.engine.base.Engine DESCRIBE `SELECT * from weather`
2021-04-01 00:06:35,109 INFO sqlalchemy.engine.base.Engine {}
2021-04-01 00:06:35,234 INFO sqla

,number,available_bike_stands,available_bikes,last_update
0,42,22,8,2021-03-08 13:44:23
1,30,16,4,2021-03-08 13:45:14
2,54,31,2,2021-03-08 13:36:34
3,108,37,3,2021-03-08 13:44:45
4,56,31,9,2021-03-08 13:44:20
5,6,15,5,2021-03-08 13:43:43
6,18,16,14,2021-03-08 13:43:29
7,32,14,16,2021-03-08 13:43:44
8,52,24,8,2021-03-08 13:37:56
9,48,25,15,2021-03-08 13:30:02


In [5]:
weather.dtypes

coord_lon                     float64
coord_lat                     float64
weather_main                   object
weather_description            object
weather_icon                   object
main_temp                     float64
main_pressure                   int64
main_humidity                   int64
main_temp_min                 float64
main_temp_max                 float64
visibility                      int64
wind_speed                    float64
wind_deg                        int64
clouds_all                      int64
dt                     datetime64[ns]
sys_type                        int64
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
city_id                         int64
city_name                      object
cod                             int64
Current Time           datetime64[ns]
dtype: object

In [6]:
# Select columns containing categorical data
weather_categorical_columns = weather[['weather_description', 'weather_icon', 'sys_country']].columns

# Convert data type to category for these columns
for column in weather_categorical_columns:
    weather[column] = weather[column].astype('category')

In [7]:
weather_datetime_columns = weather[['Current Time', 'dt', 'sys_sunset', 'sys_sunrise']].columns

# Convert data type to datetime for these columns
for column in weather_datetime_columns:
    weather[column] = weather[column].astype('datetime64[ns]')

In [8]:
bikes.sort_values('last_update', inplace=True)
weather.sort_values('dt', inplace=True)

In [9]:
weatherTrim = weather[["wind_speed", "main_temp", "main_humidity", "dt"]]
weatherTrim.head(2)

,wind_speed,main_temp,main_humidity,dt
0,4.63,283.48,62,2021-03-08 13:37:42
1,4.63,283.48,62,2021-03-08 13:38:36


In [10]:
bikes.dtypes

number                            int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
dtype: object

In [11]:
weatherTrim.dtypes

wind_speed              float64
main_temp               float64
main_humidity             int64
dt               datetime64[ns]
dtype: object

In [12]:
merged_data = pd.merge_asof(bikes, weatherTrim, left_on="last_update", right_on="dt")

In [13]:
# for inspection
merged_data.to_csv("test.csv")

In [14]:
bikes.shape

(732971, 4)

In [15]:
merged_data.shape

(732971, 8)

In [16]:
data = merged_data.loc[20:684575]
data.head(2)

,number,available_bike_stands,available_bikes,last_update,wind_speed,main_temp,main_humidity,dt
20,5,20,20,2021-03-08 13:40:07,4.63,283.48,62.0,2021-03-08 13:38:36
21,89,26,14,2021-03-08 13:42:06,4.63,283.48,62.0,2021-03-08 13:38:36


In [17]:
data.dtypes

number                            int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
wind_speed                      float64
main_temp                       float64
main_humidity                   float64
dt                       datetime64[ns]
dtype: object

In [18]:
# Create new column weekday
data['weekday'] = data['last_update'].dt.dayofweek
# Create new column hour
data['hour'] = data['last_update'].dt.hour
data.nunique()

<ipython-input-18-ea40cb292515>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weekday'] = data['last_update'].dt.dayofweek
<ipython-input-18-ea40cb292515>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hour'] = data['last_update'].dt.hour


number                      110
available_bike_stands        41
available_bikes              41
last_update              281954
wind_speed                   27
main_temp                  1001
main_humidity                24
dt                         4609
weekday                       7
hour                         24
dtype: int64

In [19]:
data

,number,available_bike_stands,available_bikes,last_update,wind_speed,main_temp,main_humidity,dt,weekday,hour
20,5,20,20,2021-03-08 13:40:07,4.63,283.48,62.0,2021-03-08 13:38:36,0,13
21,89,26,14,2021-03-08 13:42:06,4.63,283.48,62.0,2021-03-08 13:38:36,0,13
22,89,26,14,2021-03-08 13:42:06,4.63,283.48,62.0,2021-03-08 13:38:36,0,13
23,107,20,20,2021-03-08 13:42:39,4.63,283.48,62.0,2021-03-08 13:38:36,0,13
24,101,25,5,2021-03-08 13:42:39,4.63,283.48,62.0,2021-03-08 13:38:36,0,13
...,...,...,...,...,...,...,...,...,...,...
684571,116,15,15,2021-03-30 09:53:39,4.63,284.28,76.0,2021-03-30 09:53:35,1,9
684572,66,20,20,2021-03-30 09:53:42,4.63,284.28,76.0,2021-03-30 09:53:35,1,9
684573,68,27,13,2021-03-30 09:53:49,4.63,284.28,76.0,2021-03-30 09:53:35,1,9
684574,68,27,13,2021-03-30 09:53:49,4.63,284.28,76.0,2021-03-30 09:53:35,1,9


In [20]:
# X for the input data to be predicted on
X = data.drop(columns = ["available_bikes", "available_bike_stands", "last_update", "dt"])
# y for the out/goal set
y = data[["available_bikes", "available_bike_stands"]]

In [21]:
X.head(0)

,number,wind_speed,main_temp,main_humidity,weekday,hour


In [22]:
# fit the model with the 
model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier()

### _Sample Prediction_

In [23]:
# give test predictor the stand number, the wind speed, the main temp (kelvin) 
# the humidity, the weekday number, and the hour of the day
model.predict([[12, 40, 300, 80, 6, 20]])

array([[17,  3]], dtype=int64)

In [24]:
forecast = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat=53.33306&lon=-6.24889&exclude=current,minutely&appid=5cf5e19adac515b3f8510d2692fddc43")
forecast

<Response [200]>

In [25]:
forecast_data = forecast.json()
#forecast_data

In [26]:
# iterate over the json array to get the hourly data
hourly_data = 0
for k, v in forecast_data.items():
    if k == "hourly":
        hourly_data = v

for x in range(len(hourly_data)):
    hourly_data[x]

# convert the hourly data to a dataframe
df = pd.DataFrame.from_dict(pd.json_normalize(hourly_data), orient='columns')
df.head(2)

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop
0,1617231600,281.93,280.83,1021,93,280.86,0.0,75,10000,2.11,63,3.39,"[{'id': 803, 'main': 'Clouds', 'description': ...",0
1,1617235200,281.54,279.88,1021,95,280.79,0.0,82,10000,2.76,55,3.88,"[{'id': 803, 'main': 'Clouds', 'description': ...",0


In [27]:
# get rid of the unneeded rows
weather_df = df[["dt", "temp", "wind_speed", "humidity"]]
weather_df.head(2)

,dt,temp,wind_speed,humidity
0,1617231600,281.93,2.11,93
1,1617235200,281.54,2.76,95


In [28]:
df.dtypes

dt              int64
temp          float64
feels_like    float64
pressure        int64
humidity        int64
dew_point     float64
uvi           float64
clouds          int64
visibility      int64
wind_speed    float64
wind_deg        int64
wind_gust     float64
weather        object
pop             int64
dtype: object

In [29]:
# convert from unix epoch time to useable format...
from datetime import datetime, timedelta
start = datetime(1970, 1, 1)  # Unix epoch start time
weather_df['datetime'] = weather_df.dt.apply(lambda x: start + timedelta(seconds=x))

<ipython-input-29-5d99e1c0bd1c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['datetime'] = weather_df.dt.apply(lambda x: start + timedelta(seconds=x))


In [51]:
# dt is no longer needed
weather_df = weather_df.drop(columns= ["dt"])
weather_df.dtypes

KeyError: "['dt'] not found in axis"

In [52]:
weather_df = weather_df.drop(columns= ["datetime"])

In [53]:
weather_df.head(2)

,temp,wind_speed,humidity,weekday,hour
0,281.93,2.11,93,2,23
1,281.54,2.76,95,3,0


In [54]:
# Create new column weekday
weather_df['weekday'] = weather_df['datetime'].dt.dayofweek
# Create new column hour
weather_df['hour'] = weather_df['datetime'].dt.hour

KeyError: 'datetime'

In [55]:
weather_df.head(2)

,temp,wind_speed,humidity,weekday,hour
0,281.93,2.11,93,2,23
1,281.54,2.76,95,3,0


In [56]:
weather_df.nunique()

temp          47
wind_speed    43
humidity      24
weekday        3
hour          24
dtype: int64

In [57]:
weather_df.shape

(48, 5)

# create a dummy weather data set and insert a dummy station number at FIRST INDEX

In [61]:
desired_row = weather_df.iloc[1].tolist()
desired_row.insert(0, 12)

In [70]:
# give test predictor the list of user input variables (in this case, a dummy set)
prediction = model.predict([desired_row])
prediction_values = prediction[0]

#sample output
print(f"Station number {desired_row[0]}")
print("-------------------")
print(f"Predicted available bikes at  : {prediction_values[0]} ")
print(f"Predicted available bike stands : {prediction_values[1]}")

Station number 12
-------------------
Predicted available bikes at  : 15 
Predicted available bike stands : 5
